In [0]:
import os, sys, vcf, gzip
from IPython.parallel import Client
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import scipy as sp
import pandas as pd
from __future__ import division
import pymongo

In [0]:
# R_HOME = "/Library/Frameworks/R.framework/Resources"
os.environ['R_HOME'] = R_HOME
# os.environ['LD_LIBRARY_PATH'] = "%s/lib" % os.environ['R_HOME']
os.environ['LD_LIBRARY_PATH'] = "%s/lib:%s" % (os.environ['R_HOME'], os.environ['LD_LIBRARY_PATH'])
import rpy2.robjects as robjects
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2
import pandas.rpy.common as com
import scipy.stats as stats
import statsmodels.formula.api as smf
import statsmodels.api as sm
from multiprocessing import Pool
from scipy.stats.stats import pearsonr
import numbers
import warnings
import traceback

In [0]:
warnings.simplefilter(action="ignore", category=pd.io.pytables.PerformanceWarning)

In [0]:
def get_views():
#     rc = Client(profile='huge')
#     rc = Client(profile='default')
    rc = Client(profile='sge')
    dview = rc[:]
    lview = rc.load_balanced_view()
    print "Connected to %d IPython engines" % len(rc)
    return rc, dview, lview

In [0]:
rc, dview, lview = get_views()

In [0]:
rc, dview, lview = get_views()
dview['R_HOME'] = R_HOME
with dview.sync_imports():
    import os
    import sys
    import scipy.stats
    import statsmodels.formula.api
    import statsmodels.api
    from scipy.stats.stats import pearsonr
    import numpy
    import scipy
    import pandas
    import rpy2
    import numbers
    import traceback
    import socket
    import pymongo
    
def setup_engines():   
    os.environ['R_HOME'] = R_HOME
    os.environ['LD_LIBRARY_PATH'] = "%s/lib:%s" % (os.environ['R_HOME'], os.environ['LD_LIBRARY_PATH'])
#     os.environ['LD_LIBRARY_PATH'] = "%s/lib" % (os.environ['R_HOME'])
    import rpy2.robjects as robjects
    r = robjects.r
    r("source('tw_calc.R')")
    r("test=read.table('twtable', header=F)")

dview['setup_engines'] = setup_engines
setup = dview.apply(setup_engines)

In [0]:
vcftools = "~/data7/src/vcftools_0.1.12a/bin/vcftools"

    /home/cfriedline/data7/src/freebayes/bin/freebayes -L /data7/eckertlab/gypsy_indiv/bamlist.txt -f /home/cfriedline/data7/assemblies/gypsy/masurca/CA/10-gapclose/genome.ctg.fasta -v /data7/eckertlab/gypsy_indiv/gypsy_freebayes_out.vcf

In [0]:
#vcf called from freebayes
fb_vcf = "/data7/eckertlab/gypsy_indiv/gypsy_freebayes_out.vcf"

In [0]:
!$vcftools --vcf $fb_vcf

    VCFtools - v0.1.12a
    (C) Adam Auton and Anthony Marcketta 2009

    Parameters as interpreted:
        --vcf /data7/eckertlab/gypsy_indiv/gypsy_freebayes_out.vcf

    After filtering, kept 192 out of 192 Individuals
    After filtering, kept 86785 out of a possible 86785 Sites
    Run Time = 3.00 seconds

___
Outputs the allele frequency for each site in a file with the suffix ".frq". 

In [0]:
!$vcftools --freq --remove-indels --vcf $fb_vcf --out $fb_vcf

    VCFtools - v0.1.12a
    (C) Adam Auton and Anthony Marcketta 2009

    Parameters as interpreted:
        --vcf /data7/eckertlab/gypsy_indiv/gypsy_freebayes_out.vcf
        --freq
        --out /data7/eckertlab/gypsy_indiv/gypsy_freebayes_out.vcf
        --remove-indels

    After filtering, kept 192 out of 192 Individuals
    Outputting Frequency Statistics...
    After filtering, kept 79761 out of a possible 86785 Sites
    Run Time = 43.00 seconds

___
Outputs the raw allele counts for each site in a file with the suffix ".frq.count"

In [0]:
!$vcftools --counts --remove-indels --vcf $fb_vcf --out $fb_vcf

    VCFtools - v0.1.12a
    (C) Adam Auton and Anthony Marcketta 2009

    Parameters as interpreted:
        --vcf /data7/eckertlab/gypsy_indiv/gypsy_freebayes_out.vcf
        --counts
        --out /data7/eckertlab/gypsy_indiv/gypsy_freebayes_out.vcf
        --remove-indels

    After filtering, kept 192 out of 192 Individuals
    Outputting Frequency Statistics...
    After filtering, kept 79761 out of a possible 86785 Sites
    Run Time = 41.00 seconds

___
Calculates a measure of heterozygosity on a per-individual basis. Specfically, the inbreeding coefficient, F, is estimated for each individual using a method of moments. The resulting file has the suffix ".het".

In [0]:
!$vcftools --het --remove-indels --vcf $fb_vcf --out $fb_vcf

    VCFtools - v0.1.12a
    (C) Adam Auton and Anthony Marcketta 2009

    Parameters as interpreted:
        --vcf /data7/eckertlab/gypsy_indiv/gypsy_freebayes_out.vcf
        --het
        --out /data7/eckertlab/gypsy_indiv/gypsy_freebayes_out.vcf
        --remove-indels

    After filtering, kept 192 out of 192 Individuals
    Outputting Individual Heterozygosity
        Individual Heterozygosity: Only using fully diploid SNPs.
        Individual Heterozygosity: Only using biallelic SNPs.
    After filtering, kept 79761 out of a possible 86785 Sites
    Run Time = 40.00 seconds

___

Reports a p-value for each site from a Hardy-Weinberg Equilibrium test (as defined by Wigginton, Cutler and Abecasis (2005)). The resulting file (with suffix ".hwe") also contains the Observed numbers of Homozygotes and Heterozygotes and the corresponding Expected numbers under HWE.

In [0]:
!$vcftools --hardy --vcf $fb_vcf --out $fb_vcf

___
This option outputs the genotypes as a large matrix. Three files are produced. The first, with suffix ".012", contains the genotypes of each individual on a separate line. Genotypes are represented as 0, 1 and 2, where the number represent that number of non-reference alleles. Missing genotypes are represented by -1. The second file, with suffix ".012.indv" details the individuals included in the main file. The third file, with suffix ".012.pos" details the site locations included in the main file.

In [0]:
!$vcftools --remove-indels --min-alleles 2 --max-alleles 2 --mac 1 --remove-filtered-all --012 --vcf $fb_vcf --out $fb_vcf

    VCFtools - v0.1.12a
    (C) Adam Auton and Anthony Marcketta 2009

    Parameters as interpreted:
        --vcf /data7/eckertlab/gypsy_indiv/gypsy_freebayes_out.vcf
        --mac 1
        --max-alleles 2
        --min-alleles 2
        --012
        --out /data7/eckertlab/gypsy_indiv/gypsy_freebayes_out.vcf
        --remove-filtered-all
        --remove-indels

    After filtering, kept 192 out of 192 Individuals
    Writing 012 matrix files ... Done.
    After filtering, kept 30791 out of a possible 86785 Sites
    Run Time = 55.00 seconds

In [0]:
snp_quals = []
snp_dp = []
for i, rec in enumerate(vcf.Reader(open(fb_vcf))):
    if rec.INFO['TYPE'][0] == 'snp':
        snp_quals.append(rec.QUAL)
        snp_dp.append(rec.INFO['DP'])
    if i % 5000 == 0:
        print "at %d" % i

In [0]:
plt.hist(snp_quals, bins=100)
plt.title("%.2f +/- %.2f [%d, %d]" % (np.mean(snp_quals), np.std(snp_quals), np.min(snp_quals), np.max(snp_quals)))
plt.xlabel("QUAL")
plt.xlim(0, 10000)
plt.show()

In [0]:
plt.hist(snp_dp, bins=100)
plt.title("%.2f +/- %.2f [%d, %d]" % (np.mean(snp_dp), np.std(snp_dp), np.min(snp_dp), np.max(snp_dp)))
plt.xlabel("DP")
plt.xlim((0,700))
plt.show()

___
##Create file for afsource

In [0]:
def get_missing(rec):
    missing = 0.0
    for sample in rec.samples:
        if not sample.data.AO:
            missing += 1
    return missing

def is_valid_snp(rec, min_qual, missing_prob):
    if rec.QUAL < min_qual:
        return False
    t = rec.INFO['TYPE']
    if len(t) > 1:
        return False
    if t[0] != 'snp':
        return False
    if get_missing(rec)/len(rec.samples) > missing_prob:
        return False
    return True    

min_qual = 0
missing_prob = 1.0
afsource_file = "%s_qual_%d_missing_%.2f.afsource" % (fb_vcf, min_qual, missing_prob)
with open(afsource_file, "w") as o:
    for i, rec in enumerate(vcf.Reader(open(fb_vcf))):
        if is_valid_snp(rec, min_qual, missing_prob):
            err = 1
            if rec.QUAL > 0:
                err = 1/rec.QUAL
            o.write("%s\n" % "locus %s pos %d %g" % (rec.CHROM, rec.POS, err))
            for sample in rec.samples:
                ref = 0
                alt = 0
                if sample.data.RO:
                    ref = sample.data.RO
                if sample.data.AO:
                    alt = sample.data.AO
                try:
                    o.write("%s\n" % "%d %d" % (ref, alt))
                except:
                    print rec.INFO
                    print sample.data
        if i % 5000 == 0:
            print "at %d" % i

An example:

    $HOME/data7/src/afsource/af -g gypsy_freebayes_out.vcf_qual_20_missing_0.50.afsource -e 0.01 -n 1 -p gypsy_freebayes_out.vcf_qual_20_missing_0.50.afsource_e0.01_

___
##Analysis

In [0]:
hardy_file = "/data7/eckertlab/gypsy_indiv/gypsy_freebayes_out.vcf.hwe"
het_file = "/data7/eckertlab/gypsy_indiv/gypsy_freebayes_out.vcf.het"
freq_file = "/data7/eckertlab/gypsy_indiv/gypsy_freebayes_out.vcf.frq"
counts_file = "/data7/eckertlab/gypsy_indiv/gypsy_freebayes_out.vcf.frq.count"
z12_file = "/data7/eckertlab/gypsy_indiv/gypsy_freebayes_out.vcf.012"
pos_file = "/data7/eckertlab/gypsy_indiv/gypsy_freebayes_out.vcf.012.pos"
indv_file = "/data7/eckertlab/gypsy_indiv/gypsy_freebayes_out.vcf.012.indv"
trans_table = "translation_table.csv"
store_path = "/data7/cfriedline/store_gypsy.h5"

In [0]:
dview['store_path'] = store_path

In [0]:
def put(path, obj):
    s = pandas.HDFStore(store_path)
    if path in s:
        print "updating %s" % path
        s.remove(path)
    s[path] = obj
    s.close()
    
def get(path):
    s = pandas.HDFStore(store_path)
    d = None
    if path in s:
        d = s[path]
    s.close()
    return d

def in_store(path):
    s = pandas.HDFStore(store_path)
    val = False
    if path in s:
        val = True
    s.close()
    return val

def remove(path):
    s = pandas.HDFStore(store_path)
    if path in s:
        print "removing %s" % path
        s.remove(path)
    s.close()
dview['get'] = get

###Read in 012 file from vcf tools

In [0]:
z12_df = None
remove("z12_df")
if in_store('z12_df'):
    z12_df = get('z12_df')
else:
    z12_df = pd.read_csv(z12_file, header=None, sep="\t")
    z12_df = z12_df.drop(0, axis=1)
    z12_df.columns = pd.Series(z12_df.columns)-1
    put('z12_df', z12_df)

In [0]:
def get_allele_counts(series, filter_missing=True):
    P = 0.0
    Q = 0.0
    PQ = 0.0
    missing = 0.0
    c = series.value_counts()
    if 0 in c:
        P = c[0]
    if 2 in c:
        Q = c[2]
    if 1 in c:
        PQ = c[1]
    if -1 in c:
        missing = c[-1]
    missing_perc = missing/len(series)
    if filter_missing and missing_perc > 0.5:
#         print "too much missing data %g" % missing_perc
        P = PQ = Q = 0.0
    return (P, PQ, Q)

def get_correction(n):
    #for finite sample size
    return (2*n)/(2*n-1)

def get_allele_freqs(series, filter_missing=True):
    c = get_allele_counts(series, filter_missing)
    if c:
        if sum(c) == 0:
            return 0.0,0.0, 0.0, c
        else:
            P = 2*c[0] + c[1]
            Q = 2*c[2] + c[1]
            total = P + Q
            p = P/total
            q = Q/total
            maf = min(p, q)
            return maf, p, q, c 
        
def get_he(series):
    maf, p, q, c = get_allele_freqs(series)
    if p:
        He = 2 * p * q * get_correction(sum(c))
        return He
    return None

def get_ho(series):
    c = get_allele_counts(series)
    if c:
        if sum(c) == 0:
            return 0.0
        else:
            Ho = c[1]/(sum(c))
            return Ho
    return None
    
def get_Fis(df):
#     df = df.ix[:,0:10]
    #need cols b/c we might be loading modified table from h5, only want orig snp cols
    cols = [x for x in df.columns if isinstance(x, numbers.Number)]
    obs = df[cols].apply(get_ho)
    exp = df[cols].apply(get_he)
    Fis = 1 - obs/exp
    return obs, exp, Fis

In [0]:
obs = exp = Fis = None
if in_store('Fis') > 0:
    obs = get('obs')
    exp = get('exp')
    Fis = get('Fis')
else:
    obs, exp, Fis = get_Fis(z12_df)
    put('obs',obs)
    put('exp',exp)
    put('Fis',Fis)

In [0]:
x = Fis.replace([np.inf, -np.inf], np.nan).dropna()

In [0]:
plt.hist(x, bins=12)
plt.title("n=%d" % len(x))
plt.show()

###Remove based on F_is outliers

In [0]:
Fis_filter_fis = Fis[(Fis >= -0.5) & (Fis <= 0.5)] # by Fis value
print "removed", (len(Fis)-len(Fis_filter_fis))

    removed 19551

###Remove based on MAF < 0.01

In [0]:
rare = []
for snp_index in Fis_filter_fis.index:
    maf, p, q, c = get_allele_freqs(z12_df[snp_index])
    if maf < 0.01:
        rare.append(snp_index)
print len(rare)

    219

In [0]:
Fis_filter_fis_rare = Fis_filter_fis.drop(rare)

###Put filtered data back with sample/pop

In [0]:
df = None
if in_store('df'):
    df = get('df')
    z12_df = get('z12_df')
else:
    trans_df = pd.read_csv(trans_table, header=0, index_col=0, sep="\t")
    for i, line in enumerate(open(indv_file)):
        line = line.strip()
        trans = trans_df.ix[line]
        z12_df.ix[i, 'sample'] = "%s_%d" % (trans['pop'], trans['indiv'])
        z12_df.ix[i, 'pop'] = trans['pop']
        z12_df.ix[i, 'dupl'] = trans['dup']
    df = z12_df[z12_df['dupl']==0]
    df = df.drop('dupl', axis=1)
    put('df',df)
    put('z12_df',z12_df)

In [0]:
pop_id = {}
i = 0
for p in sorted(z12_df['pop'].unique()):
    pop_id[p] = i
    i+=1
pop_id

In [0]:
hierf = None
# store.remove('hierf')
if in_store('hierf'):
    hierf = get('hierf')
    df = get('df')
else:
    hierf_trans = {0:11, 1:12, 2:22, -1:'NA'}
    def apply_hierf_trans(series):
        return [hierf_trans[x] if x in hierf_trans else x for x in series]
    df = df.apply(apply_hierf_trans)

    def assign_popid(series):
        series['popid'] = pop_id[series['pop']]
        return series
    df = df.apply(assign_popid, axis=1)  

    import numbers
    hierfcols = ['sample', 'popid']
    # hierfcols.extend(df.columns.values[0:-3])
    hierfcols.extend(Fis_filter_fis_rare.index)
    hierf = df[hierfcols].sort('popid')
    cols = [None]*len(hierf.columns)
    for i, x in enumerate(hierf.columns):
        if isinstance(x, numbers.Number):
            cols[i] = "L%d" % x
        else:
            cols[i] = x
    hierf.columns = pd.Index(cols)
    hierf.to_csv("hierfstat.txt", header=True, index=False, sep="\t")
    put('hierf',hierf)
    put('df',df)

In [0]:
%%R
library(hierfstat)

In [0]:
%%R
data = read.table("hierfstat.txt", header=T, sep="\t")
levels = data.frame(data$popid)
loci = data[,3:ncol(data)]

In [0]:
%%R
res = varcomp.glob(levels=levels, loci=loci, diploid=T)

In [0]:
%%R
saveRDS(res, "hierfstat.rds")

In [0]:
%%R
res = readRDS("hierfstat.rds")

In [0]:
res = com.convert_robj(robjects.r('res'))

In [0]:
loc_df = res['loc']
F_df = res['F']
overall_df = res['overall']

In [0]:
def compute_fst(series):
    Hs = series[0]
    Ht = sum(series)
    return (Ht-Hs)/Ht

In [0]:
loci_fst = loc_df.apply(compute_fst, axis=1)

In [0]:
plt.hist(loci_fst, bins=20)
plt.title("n=%d (%.2f, %.2f)" % (len(loci_fst), np.min(loci_fst), np.max(loci_fst)))
plt.show()

___
###Setup for PCA

In [0]:
pca_data = None
if in_store('pca_data'):
    pca_data = get('pca_data')
else:
    filtered_cols = Fis_filter_fis_rare.index.tolist()
    filtered_cols.extend(['sample', 'pop','dupl'])
    pca_data = z12_df[filtered_cols]
    pca_data = pca_data[pca_data['dupl']==0]
    cols = [None]*len(pca_data.columns)
    for i, x in enumerate(pca_data.columns):
        if isinstance(x, numbers.Number):
            cols[i] = "L%d" % x
        else:
            cols[i] = x
    pca_data.columns = pd.Index(cols)
    put('pca_data', pca_data)

In [0]:
def swap_alleles(series):
#     series=pca_data[series]
    if series.name.startswith("L"):
        maf,p,q,c = get_allele_freqs(series, filter_missing=False)
        if maf == p:
            return series.replace({0:2, 2:0})
    return series

def center_and_standardize_value(val, u, var):
    if val == -1:
        return 0.0
    return (val-u)/var


def center_and_standardize(series):
#     series=pca_data[series]
    import pandas as pd
    import numpy as np
    if series.name.startswith("L"):
        maf,p,q,c = get_allele_freqs(series, filter_missing=False)
        u = sum([i*x for i, x in enumerate(c)])/sum(c)
        var = np.sqrt(maf*(1-maf))
        return series.apply(center_and_standardize_value, args=(u, var))
    return series

# dview['swap_alleles'] = swap_alleles
# dview['swap_allele'] = swap_allele
# dview['center_and_standardize'] = center_and_standardize
# dview['center_and_standardize_value'] = center_and_standardize_value
# dview['get_allele_counts'] = get_allele_counts
# dview['get_allele_freqs'] = get_allele_freqs

In [0]:
# dview.push({'pca_data':pca_data})

In [0]:
#make sure minor allele is 2
pca_maf = None
if in_store('pca_maf'):
    pca_maf = get('pca_maf')
else:
    pca_maf = pca_data.apply(swap_alleles)
    # pca_maf = pd.concat(lview.map_async(swap_alleles, pca_data.columns.tolist()).get(), axis=1)
    put('pca_maf',pca_maf)

#standardize with minor allele freq
pca_std = None
pca_std_data = None
if in_store('pca_std'):
    pca_std = get('pca_std')
    pca_std_data = get('pca_std_data')
else:
    pca_std = pca_maf.apply(center_and_standardize)
    # pca_std = pd.concat(lview.map_async(center_and_standardize, [pca_maf[col] for col in pca_maf]).get(), axis=1)
    #get only the data cols
    pca_std_data = pca_std.ix[:,0:-3]
    put('pca_std',pca_std)
    put('pca_std_data',pca_std_data)

In [0]:
for x in xrange(5):
    print pca_std_data.ix[:,x].describe()

    count    1.890000e+02
    mean     2.937098e-18
    std      9.833783e-01
    min     -3.631365e-01
    25%     -3.631365e-01
    50%      0.000000e+00
    75%      0.000000e+00
    max      5.326002e+00
    dtype: float64
    count    1.890000e+02
    mean    -1.844498e-16
    std      9.954059e-01
    min     -6.188527e-01
    25%     -6.188527e-01
    50%      0.000000e+00
    75%      0.000000e+00
    max      2.922360e+00
    dtype: float64
    count    1.890000e+02
    mean     3.289550e-17
    std      1.302756e+00
    min     -9.047280e-01
    25%     -9.047280e-01
    50%     -9.047280e-01
    75%      1.758245e+00
    max      4.421218e+00
    dtype: float64
    count    1.890000e+02
    mean     4.346905e-17
    std      1.379861e+00
    min     -9.794946e-01
    25%     -9.794946e-01
    50%     -9.794946e-01
    75%      1.552122e+00
    max      4.083739e+00
    dtype: float64
    count    1.890000e+02
    mean    -8.693810e-17
    std      1.431318e+00
    min     -1.326130e+00
    25%     -1.326130e+00
    50%      0.000000e+00
    75%      8.450828e-01
    max      3.016296e+00
    dtype: float64

In [0]:
r = robjects.r

In [0]:
prcomp = r('prcomp')
summary = r('summary')

In [0]:
prcomp_res = prcomp(pca_std_data, scale=False, center=False)

In [0]:
print summary(prcomp_res)

In [0]:
x = com.convert_robj(prcomp_res.rx2("x"))
x.index = pca_std_data.index
joined = x.join(pca_maf)

In [0]:
colors=[pop_id[x.split("_")[0]] for x in joined.sample.tolist()]
labels = [x.split("_")[0] for x in joined.sample.tolist()]
plt.scatter(joined.PC1, joined.PC2, c=colors, s=50)
fig = plt.gcf()
fig.set_size_inches(10,8)
plt.title("PCA of n=%d samples on %d loci" % (len(joined), len(pca_std_data.columns)))
plt.xlabel("PC1 (0.021%)")
plt.ylabel("PC2 (0.015%)")
plt.show()

In [0]:
def compute_missing_perc(series):
    missing = 0
    total = 0
    for x in series.index:
        if x.startswith("L"):
            if series[x] == -1:
                missing += 1
            total +=1
    return missing/total
    
missing_perc = joined.apply(compute_missing_perc, axis=1)

In [0]:
joined['missing_perc'] = missing_perc
put('joined', joined)

In [0]:
plt.scatter(joined.missing_perc, joined.PC1)
plt.xlabel("missing perc")
plt.ylabel("PC1")
fig = plt.gcf()
fig.set_size_inches(10,8)
plt.show()

In [0]:
import urllib2

In [0]:
url = 'https://docs.google.com/uc?export=download&id=0B4xHxBFoPCoWT0NneHJadUI0OHM'
response = urllib2.urlopen(url)
pheno = pd.read_excel(response, "Males-forGenomics-final")

In [0]:
pheno=pheno[['Population', 'Number', 'Mass', 'Pupual Duration', 'Total Dev Time']]
for x in pheno.index:
    pheno.ix[x, 'sample_pheno'] = "%s_%d" % (pheno.ix[x, 'Population'], pheno.ix[x, 'Number'])

In [0]:
pheno_index = []
missing = [] 
for x in pheno.index:
    sample = pheno.ix[x]['sample_pheno']
    target = joined[joined['sample']==sample]
    if len(target.index):
        i = target.index.tolist()[0]
        pheno_index.append(i)
    else:
        missing.append(x)

In [0]:
pheno = pheno.drop(missing)

In [0]:
pheno = pheno.set_index(pd.Index(pheno_index))
pheno = pheno.sort()
put('pheno', pheno)

In [0]:
jp = None
if in_store('jp'):
    jp = get('jp')
else:
    jp=pd.concat([joined,pheno], axis=1)
    jp=jp.drop(jp[np.isnan(jp['Mass'])].index)
    put('jp', jp)

___
###PCA Price et al. 2006

In [0]:
%%R 
source('tw_calc.R')
test=read.table("twtable", header=F)

In [0]:
pca_data2 = pca_data2_std = None
if in_store('pca_data2'):
    pca_data2 = get('pca_data2')
    pca_data2_std = get('pca_data2_std')
else:
    pca_data2 = jp[[x for x in jp.columns if x.startswith("L")]]
    pca_data2_std = pca_data2.apply(center_and_standardize)
    pca_data2_std.columns = [x.replace("L", "S") for x in pca_data2_std.columns]
    put('pca_data2',pca_data2)
    put('pca_data2_std',pca_data2_std)

In [0]:
r = robjects.r
prcomp = r('prcomp')
TWcalc = r('TWcalc')

In [0]:
drop = 0
S0 = pca_data2_std.drop('S%d' % drop, axis=1)

In [0]:
tw_e = tw_p = None
if in_store('tw_p'):
    tw_p = get('tw_p')
    tw_e = get('tw_e')
else:
    tw = TWcalc(com.convert_to_r_matrix(S0), 10)
    tw_p = com.convert_robj(tw.rx2(2))
    tw_e = com.convert_robj(tw.rx2(1))
    put('tw_p', pd.Series(tw_p))
    put('tw_e', pd.Series(tw_e))
print tw_e
print tw_p

In [0]:
tw_num = 0
for i, p in enumerate(tw_p):
    if p > 0.05:
        tw_num = i
        break
print "Tracy-Widom test yields %d axes of pop structure" % tw_num

In [0]:
def do_pca(df_name, col):
    import traceback
    try:
        df = get(df_name)
        df = df.drop(col, axis=1)
        prcomp = rpy2.robjects.r('prcomp')
        import pandas.rpy.common as com
        res = prcomp(com.convert_to_r_matrix(df), center=False, scale=False)
        return df_name, col, socket.gethostname(), com.convert_robj(res.rx2('x'))
    except:
        return df_name, col, socket.gethostname(), traceback.format_exc()
dview['do_pca'] = do_pca

###Do Leave-One-Out PCA for all loci

In [0]:
rc, dview, lview = get_views()
pca_jobs = []
for i, col in enumerate(pca_data2_std):
    if col.startswith("S"):
        if i % 1000 == 0:
            print "at %d" % i
#             do_pca('pca_data2_std', col)
        pca_jobs.append(lview.apply_async(do_pca, 'pca_data2_std', col))

In [0]:
write = True
pca_ready = 0
submitted = []
completed = []
pca_results = []
failed = []
for j in pca_jobs:
#     print j.stdout
    if j.ready():
        pca_ready+=1
        submitted.append(j.metadata.submitted)
        completed.append(j.metadata.completed)
        if write:
            res = None
            try:
                res = j.get()
                pca_results.append(res)
                if not isinstance(res[-1], pd.DataFrame):
                    print "failed at %s" % res
                    failed.append(res)
            except:
                pass
print "%d/%d" % (pca_ready, len(pca_jobs))
print failed

In [0]:
for i, p in enumerate(pca_results):
    if i % 1000 == 0:
        print i, p[0:3]
    node = "pca/%s" % p[1]
    put(node, p[-1])

In [0]:
print max(completed)-min(submitted)

    0:29:16.933227

###group loci by maf

In [0]:
def get_maf_bin(series):
    bins = [0.1, 0.2, 0.3, 0.4, 0.5]
    maf, p, q, c = get_allele_freqs(pca_maf[series])
    for elem in bins:
        if maf <= elem:
            return elem
maf_bins = pca_maf.groupby(get_maf_bin, axis=1)

In [0]:
loci = 0
bin_names = []
for b, pca_maf_bin in maf_bins:
    bin_cols = [x for x in pca_maf_bin.columns if x.startswith("L")]
    std_cols = [x.replace("L", "S") for x in bin_cols]
    std_bin = pca_data2_std[std_cols]
    print b, len(bin_cols), len(std_bin.columns)
    loci += len(bin_cols)
    bin_name = "pca_bin/bin_%s/df" % str(int(b*100))
    put(bin_name, std_bin)    
    bin_names.append(bin_name)
print loci

In [0]:
rc, dview, lview = get_views()
bin_jobs = {}
for bin_name in bin_names:
    print bin_name
    df = get(bin_name)
    node_path = "/".join(bin_name.split("/")[0:-1])
    bin_jobs[node_path] = []
    for i, col in enumerate(df):
        if col.startswith("S"):
            if i % 1000 == 0:
                print "at %d" % i
#             print do_pca(bin_name, col)
#             bin_jobs[node_path].append(col)
            bin_jobs[node_path].append(lview.apply_async(do_pca, bin_name, col))

In [0]:
write=True
rerun_jobs = False
failed_jobs = {}
failed = []
bin_results = []

# if not store.is_open:
#     store.open()
for node_path, jobs in bin_jobs.items():
    ready = 0
    good = 0
    local_failed = 0
    
    for j in jobs:
        if j.ready():
            res = j.get()
            if isinstance(res[-1], pd.DataFrame):
                good += 1
                if write:                    
                    bin_results.append(res)
            else:
                local_failed += 1
                failed.append(res)
            ready+=1
    print "%s: %d/%d (%d, %d)" % (node_path, ready, len(jobs), good, local_failed)
print "failed %s" % len(failed)
# if rerun_jobs:
#     failed_bins = set([x[0] for x in failed])
#     failed_dict = {}
#     for name in failed_bins:
#         if not name in failed_dict:
#             failed_dict[name] = []
#     for f in failed:
#         failed_dict[f[0]].append(f[1])
#     for bin_name, cols in failed_dict.items():
#         failed_jobs[bin_name] = []
#         df = store[bin_name]
#         node_path = "/".join(bin_name.split("/")[0:-1])
#         for col in cols:
#             if col.startswith("S"):
# #                 print col
#                 if i % 1000 == 0:
#                     print "at %d" % i
#     #             print do_pca(bin_name, col)
#     #             bin_jobs[node_path].append(col)
#                 failed_jobs[bin_name].append(lview.apply_async(do_pca, bin_name, col))

    pca_bin/bin_10: 4123/4123 (4123)
    pca_bin/bin_20: 2988/2988 (2988)
    pca_bin/bin_30: 2365/2365 (2365)
    pca_bin/bin_40: 914/914 (914)
    pca_bin/bin_50: 631/631 (631)

In [0]:
len(bin_results)

In [0]:
counts = {}
for i, res in enumerate(bin_results):
    parent = "%s" % "/".join(res[0].split("/")[0:-1])
    path = "%s/%s" % ("/".join(res[0].split("/")[0:-1]), res[1])
    
    if not parent in counts:
        counts[parent] = 1
    else:
        counts[parent] += 1
        put(path,res[-1])
    if i % 1000 == 0:
        print i, path
counts

In [0]:
concat_data = pd.concat([get(x) for x in ['pca_data2_std', 'pca_data2', 'pheno']], 
                        axis=1, 
                        join="inner")
concat_data.columns = [x.replace(" ", "_") for x in concat_data.columns]
store['concat_data'] = concat_data

In [0]:
def convert_fit_results(fit):
    import numbers
    import pandas as pd
    summary = None
    params = None
    diag=None
    curkey = None
    for i, table in enumerate(fit.summary().tables):
        if i != 1:
            sdata = {}
            for elem in table.data:
                for ind in elem:
                    ind = ind.strip()
                    if ":" in ind:
                        curkey = ind
                    else:
                        val = None
                        if isinstance(ind, numbers.Number):
                            val = float(ind)
                        else:
                            val = ind
                        sdata[curkey] = val
            if i == 0:
                summary = pd.Series(sdata)
            else:
                diag = pd.Series(sdata, dtype=float)
        else:
            params = pd.DataFrame(table.data)

    params.ix[0,0] = "x"
    params.columns=[x.replace(" ", "") for x in params.ix[0,:]]
    params = params.drop(0)
    params.index = params.ix[:,0]
    params.index.name = None
    params=params.drop("x", axis=1)
    return summary, params, diag, fit.resid
dview['convert_fit_results'] = convert_fit_results

In [0]:
rc, dview, lview = get_views()

In [0]:
def fit_model(df_path, locus, dep_var, tw_num):
    import pandas as pd
    import statsmodels.formula.api as smf
    from scipy.stats.stats import pearsonr
    import traceback
    try:
        store = pd.HDFStore(store_path)
        df = store[df_path]
        df.index = pd.Index([int(x) for x in df.index])
        m = pd.concat([get('concat_data'), df], axis=1, join="inner")
        ind = "+".join(["PC%d" % (x+1) for x in xrange(tw_num)])
        p_model = smf.ols(formula="%s ~ %s" % (dep_var, ind), data=m)
        p_fit = p_model.fit()
        g_model = smf.ols(formula="%s ~ %s" % (locus, ind), data=m)
        g_fit = g_model.fit()
        corrected_r = pearsonr(p_fit.resid, g_fit.resid)
        uncorrected_r = pearsonr(m[dep_var], m[locus])
        return {"p_fit":convert_fit_results(p_fit), "g_fit":convert_fit_results(g_fit), "r_c":corrected_r, "r_u":uncorrected_r}
    except:
        return traceback.format_exc()
dview['fit_model']=fit_model

In [0]:
def get_node_children(node_name): 
    s = pd.HDFStore(store_path)
    node = s.get_node(node_name)
    children = []
    for child, df in node._v_children.items():
        children.append(df._v_pathname)
    s.close()
    return children

In [0]:
store.close()

In [0]:
# rc, dview, lview = get_views()
# refresh=dview.apply(refresh_store).get()
# print "refreshed %d engines" % len(refresh)
rc, dview, lview = get_views()
bin_model_jobs = {}
dep_vars = ["Mass", "Pupual_Duration", "Total_Dev_Time"]
if not store.is_open:
    store.open()
for node, children in bin_nodes.items():
    node_count = 0
    bin_model_jobs[node] = []
    for df_path in children:
        locus = df_path.split("/")[-1]
        if locus.startswith("S"): #ignore the df node
            node_count += 1
            for dep_var in dep_vars:
                bin_model_jobs[node].append(lview.apply_async(fit_model, df_path, locus, dep_var, tw_num))
    print node, node_count

In [0]:
bin_model_results = {}
for k, v in bin_model_jobs.items():
    bin_model_results[k] = []
    ready = 0
    for j in v:
        if j.ready():
            ready+=1
            res = j.get()
            bin_model_results[k].append(res)
    print "%s: %d/%d" % (k, ready, len(v))

In [0]:
summaries = {}
diags = {}
params = {}
residuals = {}
for k, results in bin_model_results.items():
    for r in results:
#         print r
#         break
        p_depvar = r['p_fit'][0]['Dep. Variable:']
        g_depvar = r['g_fit'][0]['Dep. Variable:'] #g_depvar is also the locus that was excluded
        
        if not k in summaries:
            summaries[k] = pd.DataFrame(index=r['p_fit'][0].keys())
            diags[k] = pd.DataFrame(index=r['p_fit'][2].keys())
            params[k] = {}
            residuals[k] = pd.DataFrame()
            
        summaries[k]["%s_p"% g_depvar] = r['p_fit'][0]
        summaries[k]["%s_g"% g_depvar] = r['g_fit'][0]
        
        params[k]["%s_p"% g_depvar] = r['p_fit'][1]
        params[k]["%s_g"% g_depvar] = r['g_fit'][1]
        
        diags[k]["%s_p"% g_depvar] = r['p_fit'][2]
        diags[k]["%s_g"% g_depvar] = r['g_fit'][2]
        
        residuals[k]["%s_%s" % (g_depvar, p_depvar)] = r['p_fit'][3]
        residuals[k]["%s_%s" % (g_depvar, g_depvar)] = r['g_fit'][3]
#     break

In [0]:
for bin_name, df in summaries.items():
    path="model_fit/%s/summary" % (bin_name)
    print path
    store[path] = df

In [0]:
for bin_name, df_dict in params.items():
    path="model_fit/%s/params" % (bin_name)
    print path
    store[path] = pd.Panel(df_dict)

In [0]:
for bin_name, df in diags.items():
    path="model_fit/%s/diags" % (bin_name)
    print path
    store[path] = df

In [0]:
for bin_name, df in residuals.items():
    path="model_fit/%s/residuals" % (bin_name)
    print path
    store[path] = df

In [0]:
store['model_fit/pca_bin/bin_10/summary']

In [0]:
bin_model_results = {}
for k, v in bin_model_jobs.items():
    bin_model_results[k] = []
    ready = 0
    for j in v:
        if j.ready():
            ready+=1
            res = j.get()
            bin_model_results[k].append(res)
    print "%s: %d/%d" % (k, ready, len(v))

In [0]:
summaries = {}
diags = {}
params = {}
residuals = {}
r_c = {}
r_u = {}

for k, results in bin_model_results.items():
    for i, r in enumerate(results):
        if i % 1000 == 0:
            print "%s: at %d/%d" % (k, i, len(results))
#         print r
#         break
        p_depvar = r['p_fit'][0]['Dep. Variable:']
        g_depvar = r['g_fit'][0]['Dep. Variable:'] #g_depvar is also the locus that was excluded
        
        if not k in summaries:
            summaries[k] = pd.DataFrame(index=r['p_fit'][0].keys())
            diags[k] = pd.DataFrame(index=r['p_fit'][2].keys())
            params[k] = {}
            residuals[k] = pd.DataFrame()
            r_c[k] = pd.DataFrame()
            r_u[k] = pd.DataFrame()
            
        summaries[k]["%s_p"% g_depvar] = r['p_fit'][0]
        summaries[k]["%s_g"% g_depvar] = r['g_fit'][0]
        
        params[k]["%s_p"% g_depvar] = r['p_fit'][1]
        params[k]["%s_g"% g_depvar] = r['g_fit'][1]
        
        diags[k]["%s_p"% g_depvar] = r['p_fit'][2]
        diags[k]["%s_g"% g_depvar] = r['g_fit'][2]
        
        residuals[k]["%s_%s" % (g_depvar, p_depvar)] = r['p_fit'][3]
        residuals[k]["%s_%s" % (g_depvar, g_depvar)] = r['g_fit'][3]
        
        r_c[k].ix[g_depvar, p_depvar] = r['r_c'][0]
        r_u[k].ix[g_depvar, p_depvar] = r['r_u'][0]
#     break

In [0]:
for bin_name, df in summaries.items():
    path="model_fit/%s/summary" % (bin_name)
    print path
    put(path,df)

In [0]:
for bin_name, df_dict in params.items():
    path="model_fit/%s/params" % (bin_name)
    print path
    put(path, pd.Panel(df_dict))

In [0]:
for bin_name, df in diags.items():
    path="model_fit/%s/diags" % (bin_name)
    print path
    put(path, df)

In [0]:
for bin_name, df in residuals.items():
    path="model_fit/%s/residuals" % (bin_name)
    print path
    put(path, df)

In [0]:
for bin_name, df in r_c.items():
    c_path="model_fit/%s/r_c" % (bin_name)
    u_path = "model_fit/%s/r_u" % (bin_name)
    print c_path
    put(c_path, df)
    print u_path
    put(u_path, r_u[bin_name])

In [0]:
print get('model_fit/pca_bin/bin_10/r_c').ix[0:5,:]
print get('model_fit/pca_bin/bin_10/r_u').ix[0:5,:]

In [0]:
get('model_fit/pca_all/r_c')

In [0]:
store['concat_data']